# 📊 Lezione 16 — Feature Importance

## Obiettivi della lezione

In questa lezione imparerai:
- 🎯 **Cos'è** la Feature Importance e perché è fondamentale
- 📊 **MDI** (Mean Decrease Impurity) — importanza built-in degli alberi
- 🔀 **Permutation Importance** — metodo model-agnostic
- 📈 **SHAP Values** — spiegabilità avanzata del modello
- ⚖️ **Confronto** tra i diversi metodi
- 🎯 **Feature Selection** basata sull'importanza

---

## 📍 Posizione nel Corso

```
Lezione 14: Alberi Decisionali e Random Forest
Lezione 15: Gradient Boosting (XGBoost, LightGBM)
👉 Lezione 16: Feature Importance ← SEI QUI
Lezione 17: Metriche Avanzate di Valutazione
Lezione 18: Clustering (K-Means, DBSCAN)
```

---

## ❓ Perché Feature Importance?

```
┌─────────────────────────────────────────────────────────────────┐
│                   PERCHÉ CI SERVE?                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  1️⃣  INTERPRETABILITÀ                                           │
│      → Spiegare PERCHÉ il modello fa certe predizioni           │
│      → Comunicare ai stakeholder (business, management)         │
│                                                                  │
│  2️⃣  FEATURE SELECTION                                          │
│      → Eliminare features inutili o ridondanti                  │
│      → Ridurre dimensionalità e overfitting                     │
│                                                                  │
│  3️⃣  DEBUG DEL MODELLO                                          │
│      → Verificare che usi features sensate                      │
│      → Identificare data leakage                                │
│                                                                  │
│  4️⃣  DOMAIN KNOWLEDGE                                           │
│      → Scoprire insights sui dati                               │
│      → Validare ipotesi di business                             │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

# Section 1 — I Tre Metodi di Feature Importance

---

## 📊 Overview dei Metodi

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    TRE APPROCCI PRINCIPALI                               │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  1️⃣  MDI (Mean Decrease Impurity)                                       │
│      ────────────────────────────                                        │
│      • Come funziona: Misura quanto ogni feature riduce l'impurità      │
│      • Pro: Velocissimo (calcolato durante training)                    │
│      • Contro: Bias verso features con alta cardinalità                 │
│      • Dove: .feature_importances_ in sklearn                           │
│                                                                          │
│  2️⃣  PERMUTATION IMPORTANCE                                             │
│      ───────────────────────────                                         │
│      • Come funziona: Mischia una feature e vede quanto peggiora        │
│      • Pro: Model-agnostic, più affidabile                              │
│      • Contro: Più lento, sensibile a correlazioni                      │
│      • Dove: sklearn.inspection.permutation_importance                  │
│                                                                          │
│  3️⃣  SHAP VALUES                                                        │
│      ────────────                                                        │
│      • Come funziona: Teoria dei giochi (contributo marginale)          │
│      • Pro: Spiega OGNI predizione, molto accurato                      │
│      • Contro: Computazionalmente costoso                               │
│      • Dove: libreria shap                                              │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

---

## 🔢 MDI — Mean Decrease Impurity

```
Come funziona MDI:
─────────────────

Per ogni nodo dell'albero che usa feature X:
   Importance(X) += (n_samples × impurity_decrease)

Poi viene normalizzato e mediato su tutti gli alberi.

Esempio visivo:
                    ┌─────────────┐
                    │  Gini=0.5   │
                    │  Split: Age │ ← Age contribuisce a ridurre Gini
                    └──────┬──────┘
                           │
              ┌────────────┴────────────┐
              │                         │
        ┌─────▼─────┐             ┌─────▼─────┐
        │ Gini=0.3  │             │ Gini=0.2  │
        │Split:Salary│            │ Pure leaf │
        └───────────┘             └───────────┘

Age importance = 0.5 - (0.3 + 0.2)/2 = 0.25 (pesato per n_samples)
```

⚠️ **PROBLEMA MDI**: Features con molti valori unici (alta cardinalità)
tendono ad avere importanza gonfiata!

In [ ]:
# ============================================
# DEMO: MDI (Mean Decrease Impurity)
# Feature Importance built-in degli alberi
# ============================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("MDI — Mean Decrease Impurity")
print("=" * 60)

# Caricamento dataset
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

print(f"\n📊 Dataset: Breast Cancer Wisconsin")
print(f"   - Campioni: {len(X)}")
print(f"   - Features: {X.shape[1]}")

# Training Random Forest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

print(f"\n✅ Random Forest addestrato (100 alberi)")
print(f"   Accuracy: {rf.score(X_test, y_test):.4f}")

# Estrazione MDI importance
mdi_importance = pd.DataFrame({
    'Feature': X.columns,
    'MDI_Importance': rf.feature_importances_
}).sort_values('MDI_Importance', ascending=False)

print("\n🏆 Top 10 Features (MDI):")
print("-" * 40)
for i, row in mdi_importance.head(10).iterrows():
    bar = "█" * int(row['MDI_Importance'] * 50)
    print(f"   {row['Feature']:25} {row['MDI_Importance']:.4f} {bar}")

# Visualizzazione Top 15
plt.figure(figsize=(10, 6))
top15 = mdi_importance.head(15)
colors = plt.cm.Blues(np.linspace(0.3, 0.9, 15))[::-1]
plt.barh(top15['Feature'], top15['MDI_Importance'], color=colors, edgecolor='black')
plt.xlabel('MDI Importance')
plt.title('📊 Feature Importance (MDI) - Top 15')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\n💡 Nota: MDI è veloce ma può essere biased verso features con")
print("   alta cardinalità (molti valori unici).")

# Section 2 — Permutation Importance

---

## 🔀 Come Funziona

```
IDEA: Se una feature è importante, mischiarla (shuffling) 
      dovrebbe peggiorare le predizioni.

ALGORITMO:
──────────

1. Calcola score baseline sul test set
2. Per ogni feature X:
   a. Mischia casualmente i valori di X
   b. Calcola nuovo score
   c. Importance(X) = baseline_score - shuffled_score
3. Ripeti n volte per avere una stima robusta

ESEMPIO:
────────
                    Accuracy
Baseline:           0.95
Shuffle "Age":      0.70  → Importance = 0.95 - 0.70 = 0.25 ⬆️ IMPORTANTE!
Shuffle "Color":    0.94  → Importance = 0.95 - 0.94 = 0.01 ⬇️ poco importante
```

## ✅ Vantaggi della Permutation Importance

| Vantaggio | Descrizione |
|-----------|-------------|
| **Model-agnostic** | Funziona con QUALSIASI modello |
| **Affidabile** | Meno bias rispetto a MDI |
| **Post-hoc** | Calcolata dopo il training, su dati reali |
| **Interpretabile** | "Quanto peggiora se perdo questa info?" |

## ⚠️ Attenzione

- Features correlate possono entrambe sembrare poco importanti
- Più lenta di MDI (richiede multiple predizioni)

In [ ]:
# ============================================
# DEMO: Permutation Importance
# Metodo model-agnostic e affidabile
# ============================================

from sklearn.inspection import permutation_importance
import time

print("=" * 60)
print("PERMUTATION IMPORTANCE")
print("=" * 60)

# Usiamo lo stesso modello RF di prima
print(f"\n📊 Calcolando Permutation Importance su test set...")
print(f"   (10 ripetizioni per robustezza)")

start_time = time.time()
perm_importance = permutation_importance(
    rf, X_test, y_test, 
    n_repeats=10,           # Numero di shuffle per feature
    random_state=42,
    n_jobs=-1
)
tempo = time.time() - start_time
print(f"   ✅ Completato in {tempo:.2f} secondi")

# Creiamo DataFrame con mean e std
perm_df = pd.DataFrame({
    'Feature': X.columns,
    'Perm_Importance': perm_importance.importances_mean,
    'Std': perm_importance.importances_std
}).sort_values('Perm_Importance', ascending=False)

print("\n🏆 Top 10 Features (Permutation):")
print("-" * 50)
for i, row in perm_df.head(10).iterrows():
    bar = "█" * int(row['Perm_Importance'] * 100)
    print(f"   {row['Feature']:25} {row['Perm_Importance']:.4f} ± {row['Std']:.4f} {bar}")

# Visualizzazione con error bars
plt.figure(figsize=(10, 6))
top15_perm = perm_df.head(15)
colors = plt.cm.Greens(np.linspace(0.3, 0.9, 15))[::-1]
plt.barh(top15_perm['Feature'], top15_perm['Perm_Importance'], 
         xerr=top15_perm['Std'], color=colors, edgecolor='black', capsize=3)
plt.xlabel('Permutation Importance (decrease in accuracy)')
plt.title('🔀 Permutation Importance - Top 15 (con deviazione std)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\n💡 Le barre di errore mostrano la variabilità tra le 10 ripetizioni.")
print("   Features con alta std potrebbero essere meno stabili.")

# Section 3 — Confronto MDI vs Permutation

---

## ⚖️ Quando i Due Metodi Divergono

```
SCENARIO PROBLEMATICO: Features ad Alta Cardinalità
────────────────────────────────────────────────────

Esempio: Feature "ID Cliente" (unico per ogni riga)

MDI dice: "ID è MOLTO importante!" 😱
   → Perché crea split perfetti (ogni ID = una foglia)
   → Ma non generalizza! È overfitting puro.

Permutation dice: "ID è inutile" ✅
   → Perché su dati nuovi, shuffle di ID non cambia nulla
   → Correttamente identifica che ID non ha potere predittivo.
```

## 📊 Regola Pratica

| Situazione | Metodo Consigliato |
|------------|-------------------|
| Quick check, dati puliti | MDI (.feature_importances_) |
| Report finale, decisioni business | Permutation Importance |
| Features con cardinalità diverse | Permutation Importance |
| Modelli non-tree (SVM, NN) | Permutation Importance |
| Spiegare singole predizioni | SHAP (prossima sezione) |

In [ ]:
# ============================================
# DEMO: Confronto MDI vs Permutation Importance
# ============================================

print("=" * 60)
print("CONFRONTO: MDI vs PERMUTATION IMPORTANCE")
print("=" * 60)

# Merge dei due DataFrame
confronto = mdi_importance.merge(
    perm_df[['Feature', 'Perm_Importance']], 
    on='Feature'
)

# Calcolo ranking
confronto['Rank_MDI'] = range(1, len(confronto) + 1)
confronto = confronto.sort_values('Perm_Importance', ascending=False)
confronto['Rank_Perm'] = range(1, len(confronto) + 1)
confronto['Rank_Diff'] = abs(confronto['Rank_MDI'] - confronto['Rank_Perm'])

# Ordiniamo per MDI per il confronto
confronto = confronto.sort_values('MDI_Importance', ascending=False)

print("\n📊 Confronto Top 15 Features:")
print("-" * 70)
print(f"{'Feature':25} {'MDI':>8} {'Rank':>5} {'Perm':>8} {'Rank':>5} {'Δ Rank':>7}")
print("-" * 70)
for _, row in confronto.head(15).iterrows():
    delta = int(row['Rank_Diff'])
    emoji = "✅" if delta <= 2 else "⚠️" if delta <= 5 else "❌"
    print(f"{row['Feature']:25} {row['MDI_Importance']:>8.4f} {int(row['Rank_MDI']):>5} "
          f"{row['Perm_Importance']:>8.4f} {int(row['Rank_Perm']):>5} {delta:>5} {emoji}")

# Visualizzazione scatter plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter: MDI vs Permutation
ax1 = axes[0]
ax1.scatter(confronto['MDI_Importance'], confronto['Perm_Importance'], 
            alpha=0.7, s=80, c='steelblue', edgecolor='black')

# Linea di correlazione perfetta
max_val = max(confronto['MDI_Importance'].max(), confronto['Perm_Importance'].max())
ax1.plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Correlazione perfetta')

# Annotazioni per top 5
for _, row in confronto.head(5).iterrows():
    ax1.annotate(row['Feature'].split()[0], 
                 (row['MDI_Importance'], row['Perm_Importance']),
                 fontsize=9, alpha=0.8)

ax1.set_xlabel('MDI Importance')
ax1.set_ylabel('Permutation Importance')
ax1.set_title('📊 Correlazione tra MDI e Permutation')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Confronto ranking side by side
ax2 = axes[1]
top10_mdi = confronto.nsmallest(10, 'Rank_MDI')
y_pos = np.arange(10)
width = 0.35

bars1 = ax2.barh(y_pos - width/2, top10_mdi['Rank_MDI'], width, 
                  label='Rank MDI', color='#3498db', alpha=0.8)
bars2 = ax2.barh(y_pos + width/2, top10_mdi['Rank_Perm'], width, 
                  label='Rank Permutation', color='#2ecc71', alpha=0.8)

ax2.set_yticks(y_pos)
ax2.set_yticklabels(top10_mdi['Feature'])
ax2.set_xlabel('Ranking (1 = più importante)')
ax2.set_title('📊 Confronto Ranking Top 10')
ax2.legend()
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

# Calcolo correlazione
from scipy.stats import spearmanr
corr, pval = spearmanr(confronto['MDI_Importance'], confronto['Perm_Importance'])
print(f"\n📈 Correlazione di Spearman (ranking): {corr:.4f}")
print(f"   p-value: {pval:.2e}")

if corr > 0.8:
    print("   ✅ Alta correlazione: i due metodi concordano!")
elif corr > 0.5:
    print("   ⚠️ Correlazione moderata: alcune discrepanze")
else:
    print("   ❌ Bassa correlazione: investigare le differenze!")

# Section 4 — SHAP Values (Introduzione)

---

## 🎮 La Teoria dei Giochi Applicata al ML

**SHAP** = **SH**apley **A**dditive ex**P**lanations

```
IDEA CENTRALE:
──────────────
SHAP risponde alla domanda:
"Quanto ha contribuito OGNI feature alla predizione di QUESTO campione?"

È come dividere equamente le vincite tra giocatori di una squadra,
in base al loro contributo marginale.

ESEMPIO:
────────
Predizione: Probabilità cancro = 85%
Media dataset: 50%

SHAP spiega i +35%:
   worst concave points: +20%  (il più influente per QUESTO paziente)
   mean radius:          +10%
   worst perimeter:       +5%
   altri:                  0%

→ Possiamo dire PERCHÉ il modello ha dato questa predizione!
```

## 📊 Tipi di Visualizzazioni SHAP

| Tipo | Cosa Mostra |
|------|-------------|
| **Summary Plot** | Importanza globale + distribuzione impatto |
| **Beeswarm Plot** | Ogni punto = un campione |
| **Waterfall Plot** | Spiegazione di UNA predizione |
| **Dependence Plot** | Relazione feature-SHAP (come partial dependence) |

## ⚠️ Nota

SHAP richiede l'installazione della libreria: `pip install shap`

In [ ]:
# ============================================
# DEMO: SHAP Values
# Spiegabilità avanzata del modello
# ============================================

# Nota: Richiede 'pip install shap'
try:
    import shap
    SHAP_DISPONIBILE = True
except ImportError:
    SHAP_DISPONIBILE = False
    print("⚠️ SHAP non installato. Esegui: pip install shap")

if SHAP_DISPONIBILE:
    print("=" * 60)
    print("SHAP VALUES — Spiegabilità Avanzata")
    print("=" * 60)
    
    # Creiamo uno SHAP explainer per il nostro Random Forest
    print("\n📊 Creando SHAP explainer...")
    explainer = shap.TreeExplainer(rf)
    
    # Calcoliamo SHAP values per il test set
    print("   Calcolando SHAP values (potrebbe richiedere qualche secondo)...")
    shap_values = explainer.shap_values(X_test)
    
    print("   ✅ Completato!")
    
    # Per classificazione binaria, prendiamo i SHAP della classe positiva
    if isinstance(shap_values, list):
        shap_vals = shap_values[1]  # Classe positiva
    else:
        shap_vals = shap_values
    
    # 1. Summary Plot - Importanza globale
    print("\n📈 Summary Plot: mostra importanza E direzione dell'effetto")
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_vals, X_test, feature_names=X.columns, show=False)
    plt.tight_layout()
    plt.show()
    
    # 2. Bar plot - Importanza media assoluta
    print("\n📊 Bar Plot: importanza media (valore assoluto)")
    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_vals, X_test, feature_names=X.columns, 
                      plot_type="bar", show=False)
    plt.tight_layout()
    plt.show()
    
    # 3. Waterfall per singola predizione
    print("\n🔍 Waterfall Plot: spiegazione di UNA predizione specifica")
    print("   (Esempio: primo paziente del test set)")
    
    # Creiamo Explanation object per il primo campione
    sample_idx = 0
    expected_value = explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value
    
    plt.figure(figsize=(10, 6))
    shap.waterfall_plot(
        shap.Explanation(
            values=shap_vals[sample_idx],
            base_values=expected_value,
            data=X_test.iloc[sample_idx].values,
            feature_names=X.columns.tolist()
        ),
        show=False
    )
    plt.tight_layout()
    plt.show()
    
    # Interpretazione
    pred_proba = rf.predict_proba(X_test.iloc[[sample_idx]])[0][1]
    pred_class = rf.predict(X_test.iloc[[sample_idx]])[0]
    true_class = y_test.iloc[sample_idx] if hasattr(y_test, 'iloc') else y_test[sample_idx]
    
    print(f"\n📋 Analisi paziente #{sample_idx}:")
    print(f"   - Predizione: {'Benigno' if pred_class == 1 else 'Maligno'} (prob: {pred_proba:.2%})")
    print(f"   - Classe vera: {'Benigno' if true_class == 1 else 'Maligno'}")
    print(f"   - Il waterfall mostra COME si arriva a questa predizione")
    
else:
    print("\n" + "=" * 60)
    print("Per vedere SHAP in azione, installa la libreria:")
    print("   pip install shap")
    print("=" * 60)

# Section 5 — Feature Selection con Importance

---

## 🎯 Usare Feature Importance per Selezionare Features

```
WORKFLOW DI FEATURE SELECTION:
──────────────────────────────

1. Addestra modello con TUTTE le features
2. Calcola Permutation Importance
3. Rimuovi features con importanza ≈ 0 o negativa
4. Riaddestra e confronta performance

BENEFICI:
─────────
✅ Modello più semplice e interpretabile
✅ Meno overfitting (meno parametri)
✅ Training/inference più veloci
✅ Meno dati da raccogliere in produzione
```

## ⚠️ Attenzione alle Features Correlate

```
PROBLEMA:
─────────
Se A e B sono altamente correlate:
   - Shuffling di A: B "compensa", score non peggiora molto
   - Shuffling di B: A "compensa", score non peggiora molto
   → Entrambe sembrano poco importanti, ma insieme sono cruciali!

SOLUZIONE:
──────────
1. Rimuovere una delle due PRIMA di calcolare importance
2. Usare tecniche di clustering per features
3. Considerare gruppi di features correlate insieme
```

In [ ]:
# ============================================
# DEMO: Feature Selection basata su Importance
# ============================================

from sklearn.feature_selection import SelectFromModel

print("=" * 60)
print("FEATURE SELECTION CON IMPORTANCE")
print("=" * 60)

print(f"\n📊 Dataset originale: {X_train.shape[1]} features")

# Metodo 1: Soglia basata su importanza media
mean_importance = perm_df['Perm_Importance'].mean()
features_sopra_media = perm_df[perm_df['Perm_Importance'] > mean_importance]['Feature'].tolist()

print(f"\n📌 Metodo 1: Soglia = media importance ({mean_importance:.4f})")
print(f"   Features selezionate: {len(features_sopra_media)}")

# Metodo 2: Top K features
K = 10
top_k_features = perm_df.head(K)['Feature'].tolist()
print(f"\n📌 Metodo 2: Top {K} features")
print(f"   Features selezionate: {top_k_features[:5]}...")

# Metodo 3: SelectFromModel di sklearn (usa MDI)
selector = SelectFromModel(rf, threshold='median', prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

print(f"\n📌 Metodo 3: SelectFromModel (threshold='median')")
print(f"   Features selezionate: {X_train_selected.shape[1]}")

# Confronto performance: tutte le features vs top K
print("\n" + "=" * 60)
print("CONFRONTO PERFORMANCE")
print("=" * 60)

# Modello con tutte le features (già trainato)
acc_full = rf.score(X_test, y_test)

# Modello con top K features
rf_topk = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_topk.fit(X_train[top_k_features], y_train)
acc_topk = rf_topk.score(X_test[top_k_features], y_test)

# Modello con SelectFromModel
rf_selected = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_selected.fit(X_train_selected, y_train)
acc_selected = rf_selected.score(X_test_selected, y_test)

print("\n📊 Risultati:")
print("-" * 50)
print(f"   {'Modello':<30} {'Features':>10} {'Accuracy':>10}")
print("-" * 50)
print(f"   {'Tutte le features':<30} {X.shape[1]:>10} {acc_full:>10.4f}")
print(f"   {'Top 10 (Permutation)':<30} {K:>10} {acc_topk:>10.4f}")
print(f"   {'SelectFromModel (median)':<30} {X_train_selected.shape[1]:>10} {acc_selected:>10.4f}")

# Calcolo riduzione
riduzione_topk = (1 - K / X.shape[1]) * 100
perdita_topk = (acc_full - acc_topk) * 100

print(f"\n💡 Insight:")
print(f"   Con Top 10 features (-{riduzione_topk:.0f}% features):")
if perdita_topk > 0:
    print(f"   → Perdita accuracy: {perdita_topk:.2f}%")
else:
    print(f"   → Guadagno accuracy: {-perdita_topk:.2f}%!")
print(f"   → Trade-off accettabile? Dipende dal contesto!")

# Section 6 — Schema Mentale

---

## 🧠 Quale Metodo di Feature Importance Usare?

```
                    SCEGLI IL METODO GIUSTO
                    ─────────────────────────
                              │
                              ▼
            ┌─────────────────────────────────┐
            │   Hai bisogno di spiegare       │
            │   SINGOLE predizioni?           │
            └─────────────────┬───────────────┘
                              │
              ┌───────────────┴───────────────┐
              │                               │
              ▼                               ▼
            [ Sì ]                          [ No ]
              │                               │
              ▼                               ▼
         ┌─────────┐             ┌─────────────────────┐
         │  SHAP   │             │ Il modello è       │
         │ Values  │             │ tree-based?         │
         └─────────┘             └──────────┬──────────┘
                                            │
                          ┌─────────────────┴─────────────────┐
                          │                                   │
                          ▼                                   ▼
                        [ Sì ]                              [ No ]
                          │                                   │
              ┌───────────┴───────────┐                       ▼
              │                       │              ┌──────────────────┐
              ▼                       ▼              │   PERMUTATION    │
         Quick check?           Report finale?       │   IMPORTANCE     │
              │                       │              │ (model-agnostic) │
              ▼                       ▼              └──────────────────┘
         ┌─────────┐          ┌──────────────────┐
         │   MDI   │          │   PERMUTATION    │
         │ (fast)  │          │   IMPORTANCE     │
         └─────────┘          └──────────────────┘
```

## 📋 Tabella Riassuntiva

| Metodo | Velocità | Affidabilità | Model-Agnostic | Spiega Singole Pred. |
|--------|----------|--------------|----------------|---------------------|
| MDI | ⚡⚡⚡ | ⭐⭐ | ❌ | ❌ |
| Permutation | ⚡⚡ | ⭐⭐⭐ | ✅ | ❌ |
| SHAP | ⚡ | ⭐⭐⭐ | ✅ | ✅ |

# Section 7 — Esercizi Svolti

---

## 🎯 Esercizio 16.1 — Confronto Importance con XGBoost

**Obiettivo**: Calcolare e confrontare MDI e Permutation Importance su XGBoost.

**Consegna**:
1. Addestra un modello XGBoost sul dataset California Housing
2. Calcola MDI importance (built-in XGBoost)
3. Calcola Permutation Importance
4. Confronta i due ranking
5. Identifica le top 5 features secondo entrambi i metodi

In [ ]:
# ============================================
# ESERCIZIO 16.1 — SOLUZIONE COMPLETA
# Confronto Importance con XGBoost
# ============================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("ESERCIZIO 16.1 — Feature Importance con XGBoost")
print("=" * 60)

# 1. Caricamento dataset
california = fetch_california_housing()
X = pd.DataFrame(california.data, columns=california.feature_names)
y = california.target

print(f"\n📊 Dataset: California Housing")
print(f"   - Campioni: {len(X)}")
print(f"   - Features: {list(X.columns)}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Training XGBoost
print("\n🏋️ Training XGBoost...")
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    verbosity=0,
    n_jobs=-1
)
xgb_model.fit(X_train, y_train)

print(f"   ✅ XGBoost addestrato")
print(f"   R² Score: {xgb_model.score(X_test, y_test):.4f}")

# 3. MDI Importance (built-in XGBoost)
mdi_importance = pd.DataFrame({
    'Feature': X.columns,
    'MDI': xgb_model.feature_importances_
}).sort_values('MDI', ascending=False)
mdi_importance['Rank_MDI'] = range(1, len(mdi_importance) + 1)

print("\n📊 MDI Importance (built-in XGBoost):")
print("-" * 40)
for _, row in mdi_importance.iterrows():
    bar = "█" * int(row['MDI'] * 30)
    print(f"   {row['Feature']:12} {row['MDI']:.4f} {bar}")

# 4. Permutation Importance
print("\n🔀 Calcolando Permutation Importance...")
perm_result = permutation_importance(
    xgb_model, X_test, y_test,
    n_repeats=10,
    random_state=42,
    n_jobs=-1
)

perm_importance = pd.DataFrame({
    'Feature': X.columns,
    'Perm': perm_result.importances_mean,
    'Std': perm_result.importances_std
}).sort_values('Perm', ascending=False)
perm_importance['Rank_Perm'] = range(1, len(perm_importance) + 1)

print("\n📊 Permutation Importance:")
print("-" * 40)
for _, row in perm_importance.iterrows():
    bar = "█" * int(row['Perm'] * 5)
    print(f"   {row['Feature']:12} {row['Perm']:.4f} ± {row['Std']:.4f} {bar}")

# 5. Confronto ranking
confronto = mdi_importance.merge(perm_importance[['Feature', 'Perm', 'Rank_Perm']], on='Feature')
confronto = confronto.sort_values('MDI', ascending=False)

# Visualizzazione
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# MDI
ax1 = axes[0]
colors_mdi = plt.cm.Blues(np.linspace(0.3, 0.9, len(mdi_importance)))[::-1]
ax1.barh(mdi_importance['Feature'], mdi_importance['MDI'], color=colors_mdi, edgecolor='black')
ax1.set_xlabel('MDI Importance')
ax1.set_title('📊 MDI (XGBoost built-in)')
ax1.invert_yaxis()

# Permutation
ax2 = axes[1]
perm_sorted = perm_importance.sort_values('Perm', ascending=True)
colors_perm = plt.cm.Greens(np.linspace(0.3, 0.9, len(perm_sorted)))
ax2.barh(perm_sorted['Feature'], perm_sorted['Perm'], 
         xerr=perm_sorted['Std'], color=colors_perm, edgecolor='black', capsize=3)
ax2.set_xlabel('Permutation Importance')
ax2.set_title('🔀 Permutation Importance')

# Confronto ranking
ax3 = axes[2]
y_pos = np.arange(len(confronto))
width = 0.35
ax3.barh(y_pos - width/2, confronto['Rank_MDI'], width, label='Rank MDI', color='#3498db')
ax3.barh(y_pos + width/2, confronto['Rank_Perm'], width, label='Rank Perm', color='#2ecc71')
ax3.set_yticks(y_pos)
ax3.set_yticklabels(confronto['Feature'])
ax3.set_xlabel('Ranking (1 = più importante)')
ax3.set_title('📊 Confronto Ranking')
ax3.legend()
ax3.invert_yaxis()

plt.tight_layout()
plt.show()

# 6. Risultato finale
print("\n" + "=" * 60)
print("🏆 TOP 5 FEATURES")
print("=" * 60)
print("\n   Secondo MDI:         Secondo Permutation:")
print("-" * 50)
for i in range(5):
    mdi_feat = mdi_importance.iloc[i]['Feature']
    perm_feat = perm_importance.iloc[i]['Feature']
    match = "✅" if mdi_feat == perm_feat else "⚠️"
    print(f"   {i+1}. {mdi_feat:12}    {i+1}. {perm_feat:12} {match}")

---

## 🎯 Esercizio 16.2 — Il Problema delle Features Correlate

**Obiettivo**: Dimostrare come le features correlate influenzano la Permutation Importance.

**Consegna**:
1. Crea un dataset sintetico con 2 features altamente correlate (r > 0.9)
2. Addestra un modello e calcola Permutation Importance
3. Rimuovi una delle due features correlate
4. Ricalcola l'importance
5. Confronta i risultati e spiega il fenomeno

In [ ]:
# ============================================
# ESERCIZIO 16.2 — SOLUZIONE COMPLETA
# Il Problema delle Features Correlate
# ============================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("ESERCIZIO 16.2 — Features Correlate e Permutation Importance")
print("=" * 60)

# 1. Creazione dataset con features correlate
np.random.seed(42)
n_samples = 1000

# Feature base (informativa)
feature_A = np.random.randn(n_samples)

# Feature B: quasi identica ad A (correlazione ~0.95)
noise = np.random.randn(n_samples) * 0.3
feature_B = feature_A + noise

# Feature C: indipendente, anche informativa
feature_C = np.random.randn(n_samples)

# Feature D: rumore puro
feature_D = np.random.randn(n_samples)

# Target: dipende da A (o B, sono quasi uguali) e C
y = ((feature_A + feature_C) > 0).astype(int)

# Dataset completo
X_full = pd.DataFrame({
    'Feature_A': feature_A,
    'Feature_B': feature_B,
    'Feature_C': feature_C,
    'Feature_D': feature_D
})

# Calcolo correlazione
corr_AB = np.corrcoef(feature_A, feature_B)[0, 1]
print(f"\n📊 Dataset creato:")
print(f"   - Feature_A: informativa (base)")
print(f"   - Feature_B: copia rumorosa di A (corr = {corr_AB:.4f})")
print(f"   - Feature_C: informativa, indipendente")
print(f"   - Feature_D: rumore puro")

# 2. Training con tutte le features
X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.2, random_state=42)

rf_full = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_full.fit(X_train, y_train)

print(f"\n🏋️ Modello con TUTTE le features:")
print(f"   Accuracy: {rf_full.score(X_test, y_test):.4f}")

# Permutation Importance con tutte le features
perm_full = permutation_importance(rf_full, X_test, y_test, n_repeats=30, random_state=42, n_jobs=-1)
perm_full_df = pd.DataFrame({
    'Feature': X_full.columns,
    'Importance': perm_full.importances_mean,
    'Std': perm_full.importances_std
}).sort_values('Importance', ascending=False)

print("\n📊 Permutation Importance (TUTTE le features):")
print("-" * 50)
for _, row in perm_full_df.iterrows():
    bar = "█" * int(row['Importance'] * 20)
    print(f"   {row['Feature']:12} {row['Importance']:.4f} ± {row['Std']:.4f} {bar}")

# 3. Rimuoviamo Feature_B (la copia) e ritestiamo
print("\n" + "=" * 60)
print("📌 ORA RIMUOVIAMO Feature_B (la copia correlata)")
print("=" * 60)

X_reduced = X_full.drop('Feature_B', axis=1)
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(
    X_reduced, y, test_size=0.2, random_state=42
)

rf_reduced = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_reduced.fit(X_train_red, y_train_red)

print(f"\n🏋️ Modello SENZA Feature_B:")
print(f"   Accuracy: {rf_reduced.score(X_test_red, y_test_red):.4f}")

# Permutation Importance senza B
perm_red = permutation_importance(rf_reduced, X_test_red, y_test_red, n_repeats=30, random_state=42, n_jobs=-1)
perm_red_df = pd.DataFrame({
    'Feature': X_reduced.columns,
    'Importance': perm_red.importances_mean,
    'Std': perm_red.importances_std
}).sort_values('Importance', ascending=False)

print("\n📊 Permutation Importance (SENZA Feature_B):")
print("-" * 50)
for _, row in perm_red_df.iterrows():
    bar = "█" * int(row['Importance'] * 20)
    print(f"   {row['Feature']:12} {row['Importance']:.4f} ± {row['Std']:.4f} {bar}")

# 4. Visualizzazione confronto
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Con tutte le features
ax1 = axes[0]
colors = ['#e74c3c' if 'A' in f or 'B' in f else '#3498db' for f in perm_full_df['Feature']]
ax1.barh(perm_full_df['Feature'], perm_full_df['Importance'], 
         xerr=perm_full_df['Std'], color=colors, edgecolor='black', capsize=3)
ax1.set_xlabel('Permutation Importance')
ax1.set_title('📊 CON Feature_B (correlata ad A)')
ax1.invert_yaxis()

# Senza B
ax2 = axes[1]
colors_red = ['#e74c3c' if 'A' in f else '#3498db' for f in perm_red_df['Feature']]
ax2.barh(perm_red_df['Feature'], perm_red_df['Importance'], 
         xerr=perm_red_df['Std'], color=colors_red, edgecolor='black', capsize=3)
ax2.set_xlabel('Permutation Importance')
ax2.set_title('📊 SENZA Feature_B')
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

# 5. Spiegazione
print("\n" + "=" * 60)
print("💡 SPIEGAZIONE DEL FENOMENO")
print("=" * 60)

imp_A_full = perm_full_df[perm_full_df['Feature'] == 'Feature_A']['Importance'].values[0]
imp_B_full = perm_full_df[perm_full_df['Feature'] == 'Feature_B']['Importance'].values[0]
imp_A_red = perm_red_df[perm_red_df['Feature'] == 'Feature_A']['Importance'].values[0]

print(f"""
🔍 Cosa è successo:

   CON B presente:
   - Feature_A importance: {imp_A_full:.4f}
   - Feature_B importance: {imp_B_full:.4f}
   → Entrambe sembrano meno importanti perché si "compensano"!
   
   SENZA B:
   - Feature_A importance: {imp_A_red:.4f}
   → Ora A mostra la sua vera importanza!

📌 LEZIONE APPRESA:
   Quando shuffliamo A, B "copre" l'informazione persa (sono correlate).
   Quando shuffliamo B, A "copre" l'informazione persa.
   Risultato: entrambe sembrano meno importanti di quanto siano realmente!
   
⚠️ SOLUZIONE PRATICA:
   1. Prima di calcolare importance, rimuovi features molto correlate
   2. Oppure usa tecniche come clustering di features
   3. Oppure interpreta l'importance a livello di GRUPPO di features
""")

---

## 🎯 Esercizio 16.3 — Feature Selection Automatica

**Obiettivo**: Automatizzare la selezione delle features basandosi sull'importance.

**Consegna**:
1. Carica un dataset con molte features (es. digits o altro)
2. Trova il numero ottimale di features tramite validazione
3. Crea una curva accuracy vs numero features
4. Identifica il "gomito" dove aggiungere features non aiuta più
5. Addestra il modello finale con le features selezionate

In [ ]:
# ============================================
# ESERCIZIO 16.3 — SOLUZIONE COMPLETA
# Feature Selection Automatica
# ============================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("ESERCIZIO 16.3 — Feature Selection Automatica")
print("=" * 60)

# 1. Caricamento dataset digits (64 features = pixel 8x8)
digits = load_digits()
X = pd.DataFrame(digits.data, columns=[f'pixel_{i}' for i in range(64)])
y = digits.target

print(f"\n📊 Dataset: Digits (cifre scritte a mano)")
print(f"   - Campioni: {len(X)}")
print(f"   - Features: {X.shape[1]} (pixel 8x8)")
print(f"   - Classi: 0-9")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Calcolo importance per ranking
print("\n🔀 Calcolando Permutation Importance per ranking features...")
rf_base = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
rf_base.fit(X_train, y_train)

perm_result = permutation_importance(rf_base, X_test, y_test, n_repeats=5, random_state=42, n_jobs=-1)

feature_ranking = pd.DataFrame({
    'Feature': X.columns,
    'Importance': perm_result.importances_mean
}).sort_values('Importance', ascending=False)

print("   ✅ Features ordinate per importanza")

# 3. Curva accuracy vs numero features
print("\n📈 Costruendo curva Accuracy vs Numero Features...")

n_features_to_test = [1, 2, 3, 5, 8, 10, 15, 20, 25, 30, 40, 50, 64]
results = []

for n_feat in n_features_to_test:
    # Selezioniamo le top n features
    top_features = feature_ranking.head(n_feat)['Feature'].tolist()
    
    # Training con cross-validation
    rf_temp = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
    cv_scores = cross_val_score(rf_temp, X_train[top_features], y_train, cv=5)
    
    results.append({
        'n_features': n_feat,
        'mean_accuracy': cv_scores.mean(),
        'std_accuracy': cv_scores.std()
    })
    
    print(f"   {n_feat:2} features → Accuracy: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

results_df = pd.DataFrame(results)

# 4. Visualizzazione e identificazione gomito
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Curva accuracy
ax1 = axes[0]
ax1.errorbar(results_df['n_features'], results_df['mean_accuracy'], 
             yerr=results_df['std_accuracy'], marker='o', linewidth=2, 
             capsize=4, color='#3498db', markersize=8)
ax1.set_xlabel('Numero di Features')
ax1.set_ylabel('CV Accuracy')
ax1.set_title('📈 Accuracy vs Numero Features')
ax1.grid(True, alpha=0.3)

# Identificazione gomito (dove il guadagno marginale diventa <1%)
marginal_gains = results_df['mean_accuracy'].diff()
results_df['marginal_gain'] = marginal_gains

# Trova dove il guadagno marginale scende sotto 0.005 (0.5%)
threshold = 0.005
gomito_idx = None
for i, gain in enumerate(marginal_gains[1:], 1):
    if gain < threshold:
        gomito_idx = i
        break

if gomito_idx is None:
    gomito_idx = len(results_df) - 1

gomito_n_feat = results_df.iloc[gomito_idx]['n_features']
gomito_accuracy = results_df.iloc[gomito_idx]['mean_accuracy']

ax1.axvline(x=gomito_n_feat, color='red', linestyle='--', linewidth=2, 
            label=f'Gomito: {int(gomito_n_feat)} features')
ax1.legend()

# Guadagno marginale
ax2 = axes[1]
ax2.bar(results_df['n_features'][1:], marginal_gains[1:] * 100, 
        color='#2ecc71', edgecolor='black', alpha=0.8)
ax2.axhline(y=threshold*100, color='red', linestyle='--', linewidth=2, 
            label=f'Soglia: {threshold*100}%')
ax2.set_xlabel('Numero di Features')
ax2.set_ylabel('Guadagno Marginale (%)')
ax2.set_title('📊 Guadagno Marginale per Feature Aggiunta')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 5. Modello finale
print("\n" + "=" * 60)
print("🏆 MODELLO FINALE")
print("=" * 60)

# Scegliamo un punto ragionevole (es. 20 features)
n_optimal = 20  # Buon trade-off basato sulla curva
optimal_features = feature_ranking.head(n_optimal)['Feature'].tolist()

rf_final = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_final.fit(X_train[optimal_features], y_train)

acc_final = rf_final.score(X_test[optimal_features], y_test)

# Confronto con tutte le features
rf_all = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_all.fit(X_train, y_train)
acc_all = rf_all.score(X_test, y_test)

print(f"\n📊 Confronto:")
print("-" * 50)
print(f"   {'Modello':<30} {'Features':>10} {'Accuracy':>10}")
print("-" * 50)
print(f"   {'Tutte le features':<30} {64:>10} {acc_all:>10.4f}")
print(f"   {'Top 20 features':<30} {n_optimal:>10} {acc_final:>10.4f}")

perdita = (acc_all - acc_final) * 100
risparmio = (1 - n_optimal/64) * 100

print(f"\n💡 Risultato:")
print(f"   - Risparmio features: {risparmio:.1f}%")
print(f"   - Perdita accuracy: {perdita:.2f}%")

if perdita < 1:
    print(f"   ✅ Ottimo trade-off! Meno features, stessa performance.")
else:
    print(f"   ⚠️ Perdita significativa. Considerare più features.")

---

# 📚 Conclusione — Lezione 16

## 🎯 Cosa abbiamo imparato

In questa lezione abbiamo esplorato i metodi per capire **quali features sono importanti** per il modello:

### Concetti Chiave:

| Concetto | Descrizione |
|----------|-------------|
| **Feature Importance** | Misura del contributo di ogni feature alle predizioni |
| **MDI** | Mean Decrease Impurity, built-in negli alberi, veloce ma biased |
| **Permutation Importance** | Shuffle e misura peggioramento, model-agnostic |
| **SHAP** | Teoria dei giochi, spiega ogni singola predizione |
| **Feature Selection** | Rimuovere features inutili per semplificare il modello |
| **Features Correlate** | Possono mascherare la loro vera importanza |

---

## 📝 BIGNAMI — Feature Importance

```
┌─────────────────────────────────────────────────────────────────┐
│                    FEATURE IMPORTANCE                            │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  MDI (Mean Decrease Impurity)                                   │
│  ────────────────────────────                                   │
│  • model.feature_importances_                                   │
│  • Veloce (calcolato durante training)                          │
│  • ⚠️ Bias verso alta cardinalità                               │
│                                                                  │
│  PERMUTATION IMPORTANCE                                         │
│  ──────────────────────                                         │
│  • from sklearn.inspection import permutation_importance        │
│  • perm_importance(model, X_test, y_test, n_repeats=10)        │
│  • Model-agnostic, più affidabile                               │
│  • ⚠️ Lento, problemi con features correlate                   │
│                                                                  │
│  SHAP VALUES                                                    │
│  ───────────                                                    │
│  • import shap; explainer = shap.TreeExplainer(model)          │
│  • shap_values = explainer.shap_values(X)                       │
│  • Spiega OGNI predizione singolarmente                        │
│                                                                  │
├─────────────────────────────────────────────────────────────────┤
│  QUANDO USARE COSA:                                             │
│  ──────────────────                                             │
│  • Quick check           → MDI                                  │
│  • Report finale         → Permutation Importance               │
│  • Spiegare predizioni   → SHAP                                 │
│  • Modelli non-tree      → Permutation o SHAP                   │
│                                                                  │
├─────────────────────────────────────────────────────────────────┤
│  FEATURE SELECTION:                                             │
│  ──────────────────                                             │
│  • SelectFromModel(model, threshold='median')                   │
│  • Top K features (manuale)                                     │
│  • Curva accuracy vs n_features → trova il "gomito"            │
│                                                                  │
├─────────────────────────────────────────────────────────────────┤
│  ⚠️ ATTENZIONE FEATURES CORRELATE:                              │
│  ─────────────────────────────────                              │
│  Se A e B sono correlate:                                       │
│     → Shuffle A: B compensa → A sembra poco importante          │
│     → Shuffle B: A compensa → B sembra poco importante          │
│  SOLUZIONE: Rimuovere duplicati prima di calcolare importance   │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

---

## 🔗 Prossima Lezione

**Lezione 17**: Metriche Avanzate di Valutazione
- Precision, Recall, F1-Score
- Matrice di confusione
- Curve ROC e AUC
- Curva Precision-Recall
- Soglia di classificazione ottimale